## 손실 함수

손실함수란 신경망이 얼마나 맞추지 못했냐 라는 지표이다.
- 즉 낮을수록 좋은 값이다.
- 손실 함수에 영향을 미치는 것은 가중치(W)와 편향(B)
- 신경망의 학습이랑 손실 함수의 값이 낮은 가중치와 편향을 구하는 것이다.

## MSE
평균 제곱 오차

In [ ]:
import numpy as np


# 경우 1)
y = [0.1, 0.05, 0.6, 0.0, 0.05, 0.1, 0.0, 0.1, 0.0, 0.0]  
# 모델이 0번 클래스라고 예측한 확률 : 0.1
# 모델이 1번 클래스라고 예측한 확률 : 0.05
# -> 모델이 2로 예측했을 확률 : 0.6

t = [0, 0, 1, 0, 0, 0, 0, 0, 0, 0]
# 실제 정답은 2이다.

In [ ]:
def mean_squared_error(y, t):
  return 0.5 * np.sum((y - t)**2)

In [ ]:
print("MSE : {}".format(mean_squared_error(np.array(y), np.array(t))))

MSE : 0.09750000000000003


In [ ]:
# 경우 2)
# 모델이 7로 예측 했을 확률이 0.6
y_error = [0.1, 0.05, 0.1, 0.0, 0.05, 0.1, 0.0, 0.6, 0.0, 0.0]

print("MSE : {}".format(mean_squared_error(np.array(y_error), np.array(t))))

MSE : 0.5975


In [ ]:
# 아까보다 MSE 값이 훨씬 커졌다.
# 잘못 예측하니까 커짐

# CEE (Cross Entropy Error)
- 교차 엔트로피 에러

In [ ]:
def cross_entropy_error(y, t):
  delta = 1e-7  # 아주 작은 값
  return -np.sum(t * np.log(y + delta))  
# 델타를 더해주는 이유는? 
# log(0) : -np.inf -> 발산하게 하지 않기 위해 아주 작은 값을 더해줌.

In [ ]:
y = np.array([0.1, 0.05, 0.6, 0.0, 0.05, 0.1, 0.0, 0.1, 0.0, 0.0])
# 정답을 2로 예측햇을 확률이 0.6
y_error = np.array([0.1, 0.05, 0.1, 0.0, 0.05, 0.1, 0.0, 0.6, 0.0, 0.0])
# 정답을 7로 예측했을 확률이 0.6
t = np.array([0, 0, 1, 0, 0, 0, 0, 0, 0, 0])

print("y의 CEE : {:.3f}".format(cross_entropy_error(y, t)))
print("y_error의 CEE : {:.3f}".format(cross_entropy_error(y_error, t)))
# MSE보다 cross entropy가 더 결과를 극대화시킴

y의 CEE : 0.511
y_error의 CEE : 2.303


In [ ]:
# t -> one-hot encoding되있음
# cross entropy를 하고 싶으면 원핫 인코딩이 된 값 대입

In [ ]:
from tensorflow.keras import datasets
mnist = datasets.mnist

(X_train, y_train), (X_test, y_test) = mnist.load_data()

11493376/11490434 [==============================] - 0s 0us/step


In [ ]:
X_train[0].shape, y_train[0].shape

((28, 28), ())

In [ ]:
y_train.shape

(60000,)

# y_train을 OneHotEncoding 하세요
- 목표 : (60000, 10)

In [ ]:
y_train_step_1 = y_train.reshape(-1, 1)
y_train_step_1.shape

(60000, 1)

In [ ]:
y_train_step_1[:3]

array([[5],
       [0],
       [4]], dtype=uint8)

In [ ]:
from sklearn.preprocessing import OneHotEncoder
y_train_dummy = OneHotEncoder().fit_transform(y_train_step_1)
y_train_dummy = y_train_dummy.toarray()

y_train_dummy.shape

(60000, 10)

In [ ]:
y_train_dummy[:3]

array([[0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0.]])

In [ ]:
X_train = X_train.reshape(60000, -1)
X_train.shape

(60000, 784)

In [ ]:
X_train.shape, y_train_dummy.shape

((60000, 784), (60000, 10))

# 미니 배치
- 랜덤하게 뽑은 인덱스로 배치를 만든 것

In [ ]:
# 60000장의 데이터 중에 무작위로 10개만 뽑아서 미니 배치를 만들자.
# choice(범위 숫자, 갯수)

train_size = X_train.shape[0]
batch_size = 10
batch_mask = np.random.choice(train_size, batch_size)  # 60000개 중에 10개를 무작위로 뽑는다.

print(batch_mask)

[23592 26549 21131 35275  7849  5815 28583 32552  1510 32838]


In [ ]:
# 마스크에 의해서 선택된 데이터 뽑기
X_batch = X_train[batch_mask]
t_batch = y_train_dummy[batch_mask]  # target

# 배치용 크로스 엔트로피 구현하기 v.1

In [ ]:
def cross_entropy_error_v1(y, t):

  if y.ndim == 1:  # batch를 쓰지 않는 경우. 즉 1장만 검사하는 경우
    t = t.reshape(1, t.size)
    y = y.reshape(1, y.size)
  
  batch_size = y.shape[0]
  return -np.sum(t * np.log(y)) / batch_size  # OneHotEncoding된 레이블에만 사용가능
  # t,y 사이즈 동일
  # - 각 배치에 대한 모든 합 / 배치 사이즈

  # 배치 마다의 오차를 전부 더해서 배치 크기로 나눠준다.

# 배치용 크로스 엔트로피 구현하기 v.2

In [ ]:
def cross_entropy_error_v2(y, t):
  
  if y.ndim == 1:  # 데이터가 1개가 들어왔더라도 배치처럼 행동하게끔.. # 1개의 배치가 들어왔다고 생각하게끔..
    t = t.reshape(1, t.size)
    y = y.reshape(1, y.size)
  
  batch_size = y.shape[0]
  return -np.sum(np.log(y[np.arange(batch_size), t])) / batch_size  # OneHotEncoding 안 되었을 때
  # 스파스 카테고리컬 크로스 엔트로피